<a href="https://colab.research.google.com/github/DajanaSanchez/BirdMigrationClimateAnalysis/blob/main/bird_land_climate_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import gc

# ✅ Load Bird + Land Data
df_merged = pd.read_parquet("/content/drive/MyDrive/bird_land_habitat_updated.parquet")

df_merged.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,geometry,index_right,Year_right,Date_right,Latitude_right,Longitude_right,Region,Land_Cover_Type,Land_Cover_Description,distance,Year_Diff,Has_Habitat_Change
6031513,32.793335,-97.492590,1998-12-01,1998,b'\x01\x01\x00\x00\x00\x8b\x155\x98\x86_X\xc0\...,2407646,2001,2001-01,32.776141,-97.489575,US,10,Grasslands,2301.304418,3,False
1941428,49.498783,-115.793117,2019-08-01,2019,b'\x01\x01\x00\x00\x00T\x1a\x8cl\xc2\xf2\\\xc0...,57030111,2019,2019-01,49.500736,-115.804837,Canada,8,Woody Savannas,1347.001098,0,False
6792659,46.161820,-118.955240,2018-01-01,2018,"b'\x01\x01\x00\x00\x00\xc5\xb4\xca\xa7""\xbd]\x...",53930540,2018,2018-01,46.145803,-118.957465,US,12,Croplands,2585.918796,0,False
5713478,41.588694,-80.406360,2019-03-01,2019,b'\x01\x01\x00\x00\x000\xda\x88\xcc\x01\x1aT\x...,57409860,2019,2019-01,41.589102,-80.425354,US,8,Woody Savannas,2115.347150,0,False
785999,33.574042,-111.537758,2022-02-01,2022,b'\x01\x01\x00\x00\x00p\xebn\x9ej\xe2[\xc0\xfe...,64313560,2022,2022-01,33.577320,-111.551293,US,7,Open Shrublands,1569.105188,0,True
6321896,38.550381,-77.295449,2016-04-01,2016,b'\x01\x01\x00\x00\x00\x1ei\x15\xa2\xe8RS\xc0(...,47775539,2016,2016-01,38.534610,-77.272727,US,13,Urban and Built-up,3381.691859,0,True
6352904,42.870838,-72.562380,2023-03-01,2023,b'\x01\x01\x00\x00\x00`\x9bC\x08\xfe#R\xc0F\x1...,70388607,2023,2023-01,42.891016,-72.568807,US,4,Deciduous Broadleaf Forest,3147.848682,0,False
4046248,38.963910,-74.961590,2021-10-01,2021,b'\x01\x01\x00\x00\x00I\x8a\xc8\xb0\x8a\xbdR\x...,67315440,2021,2021-01,38.985272,-74.970809,US,0,Water,3226.455208,0,False
4797167,40.740104,-84.044552,2015-06-01,2015,b'\x01\x01\x00\x00\x00\x95\xfe\xca\xef\xd9\x02...,44409607,2015,2015-01,40.737850,-84.028357,US,12,Croplands,1832.964174,0,False
6081357,37.374595,-77.362896,2023-02-01,2023,b'\x01\x01\x00\x00\x000ww\xae9WS\xc0\xf4\xdes\...,70652401,2023,2023-01,37.382916,-77.372811,US,13,Urban and Built-up,1605.310210,0,False


In [ ]:
#dropping unneccessary columns
df_birdland_final = df_merged.drop(columns=['geometry', 'index_right', 'Year_right', 'Date_right', 'Latitude_right', 'Longitude_right', 'distance', 'Year_Diff'])

In [ ]:
df_birdland_final.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,Region,Land_Cover_Type,Land_Cover_Description,Has_Habitat_Change
6514229,46.707971,-123.987783,2023-05-01,2023,US,2,Evergreen Broadleaf Forest,True
4994686,43.637568,-73.431367,2020-11-01,2020,US,4,Deciduous Broadleaf Forest,False
5634323,40.565945,-75.986072,2015-09-01,2015,US,4,Deciduous Broadleaf Forest,False
3546240,38.845460,-90.147774,2021-07-01,2021,US,13,Urban and Built-up,False
1099030,39.549225,-105.075753,2015-02-01,2015,US,13,Urban and Built-up,False
294120,44.656184,-66.773561,2023-08-01,2023,US,5,Mixed Forests,False
2084857,31.651697,-81.393436,2018-04-01,2018,US,8,Woody Savannas,False
3685597,45.573228,-94.554650,2017-07-01,2017,US,12,Croplands,False
2343530,39.703251,-95.729004,2024-01-01,2024,US,12,Croplands,False
4368740,42.300393,-75.467365,2004-10-01,2004,US,8,Woody Savannas,False


In [ ]:
print(df_birdland_final.count())

Latitude_left             7498836
Longitude_left            7498836
Date_left                 7498836
Year_left                 7498836
Region                    7498836
Land_Cover_Type           7498836
Land_Cover_Description    7498836
Has_Habitat_Change        7498836
dtype: int64


In [ ]:
# ✅ Convert to GeoDataFrame
bird_land_gdf = gpd.GeoDataFrame(
    df_birdland_final,
    geometry=gpd.points_from_xy(df_merged.Longitude_left, df_merged.Latitude_left),
    crs="EPSG:4326"
)

# ✅ Drop unneeded columns (geometry in binary format)
bird_land_gdf = bird_land_gdf.drop(columns=["geometry"], errors="ignore")

In [ ]:
bird_land_gdf.count()

,0
Latitude_left,7498836
Longitude_left,7498836
Date_left,7498836
Year_left,7498836
Region,7498836
Land_Cover_Type,7498836
Land_Cover_Description,7498836
Has_Habitat_Change,7498836


In [125]:
# ✅ Load Climate Data from Parquet
climate_csv = "/content/drive/MyDrive/climate_data/climate_merged_final.csv"

# Convert 'climate_merged_final_.csv' to Parquet
climate_parquet = pd.read_csv(climate_csv)

# Save as Parquet
climate_parquet.to_parquet("climate_merged_final.parquet", engine="pyarrow", index=False)

#drop unnecessary fields
final_climate_parquet = climate_parquet.drop(columns=['Station_ID', 'Elevation', 'State', 'StationName', 'TempFlags', 'Precip_MeasurementFlag', 'Precip_QCFlag', 'Precip_SourceFlag', 'Precip_SourceIndex', 'Precip_FirstYear', 'Precip_LastYear'])

final_climate_parquet.head()

<ipython-input-125-68c66a6b0a63>:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  climate_parquet = pd.read_csv(climate_csv)


,Date,Latitude,Longitude,Temp_celsius,Precip_mm
0,1992-01,48.9333,-123.75,-99.99,428.4
1,1993-01,48.9333,-123.75,-99.99,170.1
2,1994-01,48.9333,-123.75,6.34,139.7
3,1995-01,48.9333,-123.75,4.51,225.4
4,1996-01,48.9333,-123.75,3.40,242.6


In [127]:
final_climate_parquet['Date'].max()

'2024-12'

In [128]:
# ✅ Round Latitude & Longitude to 6 decimal places
final_climate_parquet["Latitude"] = final_climate_parquet["Latitude"].round(6)
final_climate_parquet["Longitude"] = final_climate_parquet["Longitude"].round(6)

In [129]:
# ✅ Convert to GeoDataFrame
climate_gdf = final_climate_parquet[["Latitude", "Longitude", "Temp_celsius", "Precip_mm", "Date"]].copy()


# ✅ Convert Date to datetime format and extract Year
climate_gdf["Date"] = pd.to_datetime(climate_gdf["Date"], errors="coerce")
climate_gdf["Year_climate"] = climate_gdf["Date"].dt.year

# ✅ Convert to GeoDataFrame
climate_gdf = gpd.GeoDataFrame(
    climate_gdf,
    geometry=gpd.points_from_xy(climate_gdf.Longitude, climate_gdf.Latitude),
    crs="EPSG:4326"
)

# ✅ Convert numeric fields to float32 for memory efficiency
climate_gdf["Temp_celsius"] = climate_gdf["Temp_celsius"].astype("float32")
climate_gdf["Precip_mm"] = climate_gdf["Precip_mm"].astype("float32")

# ✅ Drop unnecessary columns (ensures better performance)
climate_gdf = climate_gdf.drop(columns=["geometry"], errors="ignore")

print(f"✅ Loaded Climate Data with {len(climate_gdf)} records.")

✅ Loaded Climate Data with 2160706 records.


In [130]:
climate_gdf.head()

,Latitude,Longitude,Temp_celsius,Precip_mm,Date,Year_climate
0,48.9333,-123.75,-99.989998,428.399994,1992-01-01,1992
1,48.9333,-123.75,-99.989998,170.100006,1993-01-01,1993
2,48.9333,-123.75,6.340000,139.699997,1994-01-01,1994
3,48.9333,-123.75,4.510000,225.399994,1995-01-01,1995
4,48.9333,-123.75,3.400000,242.600006,1996-01-01,1996


In [135]:
import os
import pandas as pd
import numpy as np
import gc
from scipy.spatial import cKDTree

# ✅ Define Paths
bird_land_file = "/content/drive/MyDrive/bird_land_habitat_updated.parquet"
climate_file = "/content/drive/MyDrive/climate_data/climate_merged_final.csv"
output_dir = "/content/drive/MyDrive/bird_land_climate_chunks"
os.makedirs(output_dir, exist_ok=True)  # ✅ Ensure the output directory exists

chunk_size = 250000  # ✅ Process Bird + Land data in chunks

print("✅ Loading full Bird + Land dataset...")

# ✅ Load Bird + Land Data (ENSURE ALL NECESSARY FIELDS ARE INCLUDED)
bird_land_df = pd.read_parquet(
    bird_land_file,
    columns=["Latitude_left", "Longitude_left", "Date_left", "Year_left", "Region",
             "Land_Cover_Type", "Land_Cover_Description", "Has_Habitat_Change"]
)


# Convert climate file to Parquet
climate_parquet = "climate_merged_final.parquet"  # File path, not DataFrame
pd.read_csv(climate_file).to_parquet(climate_parquet, engine="pyarrow", index=False)

# ✅ Load Climate Data
climate_df = pd.read_parquet(climate_parquet)  # Use the file path, not the DataFrame


# ✅ Convert Climate Date & Extract Year
climate_df["Date"] = pd.to_datetime(climate_df["Date"], errors="coerce")
climate_df["Year_climate"] = climate_df["Date"].dt.year
climate_df["Date_ordinal"] = climate_df["Date"].map(lambda d: d.toordinal())  # Convert Date to numerical format

print(f"✅ Climate data loaded with {len(climate_df)} records.")

# ✅ Build KDTree for Climate Data (Based on Latitude, Longitude, and Date)
climate_tree = cKDTree(climate_df[["Longitude", "Latitude", "Date_ordinal"]].values)

print("✅ KDTree built for Climate Data.")

# ✅ Define function to find the nearest climate match using `k=1`
def get_nearest_climate(row):
    """
    Finds the nearest climate record for a given bird/land observation using k=1.
    Ensures the closest match in both space and time.
    """
    # Query KDTree for **only the closest** climate record
    _, idx = climate_tree.query([row["Longitude_left"], row["Latitude_left"], row["Date_ordinal"]], k=1)

    # Retrieve the closest match
    best_match = climate_df.iloc[idx].copy()

    # Compute year difference
    best_match["Year_Diff_Climate"] = abs(best_match["Year_climate"] - row["Year_left"])

    return pd.Series({
        "Latitude_climate": best_match["Latitude"],
        "Longitude_climate": best_match["Longitude"],
        "Date_climate": best_match["Date"],
        "Year_climate": best_match["Year_climate"],
        "Temp_celsius": best_match["Temp_celsius"],
        "Precip_mm": best_match["Precip_mm"],
        "Year_Diff_Climate": best_match["Year_Diff_Climate"]
    })

print("✅ Starting chunked processing...")

# ✅ Process Bird + Land in chunks manually
for i in range(0, len(bird_land_df), chunk_size):

    chunk_file = f"bird_land_climate_chunk_{i}.parquet"
    chunk_path = os.path.join(output_dir, chunk_file)

    print(f"🔹 Processing rows {i} to {i + chunk_size}...")

    # ✅ Select chunk of Bird + Land data
    chunk = bird_land_df.iloc[i: i + chunk_size].copy()

    # ✅ Convert Date & Extract Year
    chunk["Date_left"] = pd.to_datetime(chunk["Date_left"], errors="coerce")
    chunk["Year_left"] = chunk["Date_left"].dt.year
    chunk["Date_ordinal"] = chunk["Date_left"].map(lambda d: d.toordinal())  # Convert Date to numerical format

    # ✅ Apply KDTree nearest neighbor search (Finds closest match)
    climate_info = chunk.apply(get_nearest_climate, axis=1)

    # ✅ Merge climate data back to Bird + Land data
    bird_land_climate_chunk = pd.concat([chunk, climate_info], axis=1)

    print(f"✅ Spatial join complete. {len(bird_land_climate_chunk)} records after KDTree join.")

    # ✅ Save Each Chunk to Google Drive
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    bird_land_climate_chunk.to_parquet(chunk_path, index=False)
    print(f"✅ Processed and saved {len(bird_land_climate_chunk)} rows to {chunk_path}.")

    # ✅ Free Memory
    del chunk, climate_info, bird_land_climate_chunk
    gc.collect()

print("✅ Processing complete for all chunks.")

✅ Loading full Bird + Land dataset...


<ipython-input-135-a4e9406aaece>:27: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(climate_file).to_parquet(climate_parquet, engine="pyarrow", index=False)


✅ Climate data loaded with 2160706 records.
✅ KDTree built for Climate Data.
✅ Starting chunked processing...
🔹 Processing rows 0 to 250000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_0.parquet.
🔹 Processing rows 250000 to 500000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_250000.parquet.
🔹 Processing rows 500000 to 750000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_500000.parquet.
🔹 Processing rows 750000 to 1000000...
✅ Spatial join complete. 250000 records after KDTree join.
✅ Processed and saved 250000 rows to /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_750000.parquet.
🔹 Processin

In [136]:
import os
import pandas as pd
import gc

# ✅ Define Paths
chunk_dir = "/content/drive/MyDrive/bird_land_climate_chunks"
final_output_file = "/content/drive/MyDrive/bird_land_climate_final.parquet"

print("✅ Checking for chunk files...")

# ✅ Get list of all chunked files
all_chunks = sorted([os.path.join(chunk_dir, f) for f in os.listdir(chunk_dir) if f.endswith(".parquet")])

print(f"✅ Found {len(all_chunks)} chunk files.")

if len(all_chunks) == 0:
    print("❌ No chunk files found. Ensure chunk processing is complete before merging.")
else:
    print("✅ Starting chunk merge...")

    # ✅ Initialize an empty list to store DataFrames
    merged_dfs = []

    # ✅ Read each chunk, merge them in memory-efficient way
    for chunk_file in all_chunks:
        print(f"🔹 Merging {chunk_file}...")

        # ✅ Read chunk
        df_chunk = pd.read_parquet(chunk_file)

        # ✅ Append to list
        merged_dfs.append(df_chunk)

        # ✅ Free Memory
        del df_chunk
        gc.collect()

    # ✅ Concatenate all chunks into final DataFrame
    final_df = pd.concat(merged_dfs, ignore_index=True)

    # ✅ Save the final dataset to Google Drive
    final_df.to_parquet(final_output_file, index=False)

    print(f"✅ Final dataset saved at: {final_output_file}")

    # ✅ Free Memory
    del final_df, merged_dfs
    gc.collect()

print("✅ Chunk merging complete!")

✅ Checking for chunk files...
✅ Found 30 chunk files.
✅ Starting chunk merge...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_0.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1000000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1250000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1500000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_1750000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_2000000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_2250000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_250000.parquet...
🔹 Merging /content/drive/MyDrive/bird_land_climate_chunks/bird_land_climate_chunk_2500000.parquet...
🔹 Merging /content

In [137]:
df = pd.read_parquet(final_output_file)

In [139]:
df.sample(10)

,Latitude_left,Longitude_left,Date_left,Year_left,Region,Land_Cover_Type,Land_Cover_Description,Has_Habitat_Change,Date_ordinal,Latitude_climate,Longitude_climate,Date_climate,Year_climate,Temp_celsius,Precip_mm,Year_Diff_Climate
3269642,33.884891,-78.567736,2024-04-01,2024,US,8,Woody Savannas,False,738977,33.8161,-78.7206,2024-04-01,2024,17.79,89.2,0
2445415,39.317970,-76.636280,2024-11-01,2024,US,13,Urban and Built-up,False,739191,39.2814,-76.6111,2024-11-01,2024,12.66,55.4,0
4412929,41.969694,-80.558317,2023-10-01,2023,US,0,Water,False,738794,41.7794,-80.6969,2023-10-01,2023,12.92,50.6,0
2970704,46.373217,-94.297462,2022-07-01,2022,US,8,Woody Savannas,False,738337,46.4019,-94.1272,2022-07-01,2022,21.46,56.4,0
2802133,42.678007,-82.852142,2024-12-01,2024,US,9,Savannas,False,739221,42.4072,-83.0089,2024-12-01,2024,1.95,79.0,0
5554049,32.703831,-80.600719,2024-12-01,2024,US,8,Woody Savannas,True,739221,32.8994,-80.0408,2024-12-01,2024,10.78,62.1,0
2598538,44.011210,-69.244246,2024-09-01,2024,US,5,Mixed Forests,False,739130,43.9639,-69.7114,2024-09-01,2024,16.95,46.8,0
2260116,38.120243,-75.793605,2013-12-01,2013,US,8,Woody Savannas,False,735203,38.3408,-75.5133,2013-12-01,2013,5.50,81.0,0
3818824,41.016011,-74.745018,2024-03-01,2024,US,4,Deciduous Broadleaf Forest,False,738946,41.0092,-74.7364,2024-03-01,2024,7.28,85.4,0
1742950,42.422200,-88.319700,2017-10-01,2017,US,12,Croplands,False,736603,42.5953,-87.9383,2017-10-01,2017,14.16,124.5,0


In [140]:
#rename columns
df = df.rename(columns={
    'Latitude_left': 'Latitude',
    'Longitude_left': 'Longitude',
    'Date_left': 'Observation_Date',
    'Year_left': 'Year'})

In [145]:
#pull min and max Year_climate
print(df['Year_climate'].min())
print(df['Year_climate'].max())

#pull count by Year_Diff_Climate that is greater than 0
print(df[df['Year_Diff_Climate'] > 0]['Year_Diff_Climate'].value_counts())

1974
2024
Series([], Name: count, dtype: int64)


In [146]:
#drop columns
df = df.drop(columns=['Land_Cover_Type', 'Date_ordinal', 'Latitude_climate', 'Longitude_climate', 'Date_climate', 'Year_climate', 'Year_Diff_Climate'])

In [148]:
df.sample(10)

,Latitude,Longitude,Observation_Date,Year,Region,Land_Cover_Description,Has_Habitat_Change,Temp_celsius,Precip_mm
397682,39.936948,-104.752458,2022-01-01,2022,US,Grasslands,False,-0.55,19.9
6023002,36.918298,-76.054230,2015-11-01,2015,US,Water,False,-99.99,111.9
1604123,39.874966,-86.306201,2024-06-01,2024,US,Croplands,False,23.72,80.0
7269888,33.219315,-111.892638,2023-01-01,2023,US,Open Shrublands,False,12.15,26.1
204994,50.568193,-113.703690,2011-04-01,2011,Canada,Croplands,True,2.59,68.5
7370152,37.881609,-122.142533,2020-02-01,2020,US,Savannas,True,-99.99,0.0
706266,41.248580,-72.902930,2018-09-01,2018,US,Urban and Built-up,False,-99.99,207.4
941466,29.138550,-82.169119,2023-01-01,2023,US,Grasslands,False,17.20,31.0
313930,38.554609,-121.256104,2013-02-01,2013,US,Urban and Built-up,False,-99.99,6.6
1073420,45.964018,-65.267793,2016-08-01,2016,US,Mixed Forests,False,17.62,83.8


In [155]:
df.describe()

,Latitude,Longitude,Observation_Date,Year,Temp_celsius,Precip_mm,Month
count,7.498836e+06,7.498836e+06,7498836,7.498836e+06,7.498836e+06,7.498836e+06,7.498836e+06
mean,4.238773e+01,-9.358986e+01,2019-06-21 17:59:32.958363136,2.019055e+03,9.935046e+00,8.072112e+01,5.997464e+00
min,1.934370e+01,-1.790633e+02,1974-01-01 00:00:00,1.974000e+03,-2.975000e+01,0.000000e+00,1.000000e+00
25%,3.912949e+01,-1.140327e+02,2017-05-01 00:00:00,2.017000e+03,3.850000e+00,3.600000e+01,3.000000e+00
50%,4.240090e+01,-8.596323e+01,2020-11-01 00:00:00,2.020000e+03,9.941860e+00,6.760000e+01,5.000000e+00
75%,4.621175e+01,-7.676376e+01,2023-01-01 00:00:00,2.023000e+03,1.640000e+01,1.080000e+02,9.000000e+00
max,7.315889e+01,1.792826e+02,2024-12-01 00:00:00,2.024000e+03,3.930000e+01,1.323200e+03,1.200000e+01
std,6.207601e+00,2.055219e+01,NaN,5.087835e+00,8.348045e+00,6.630785e+01,3.617435e+00


In [154]:
import pandas as pd

# ✅ Convert Date to datetime
df["Observation_Date"] = pd.to_datetime(df["Observation_Date"], errors="coerce")

# ✅ Extract Month
df["Month"] = df["Observation_Date"].dt.month

# ✅ Compute Monthly Mean (Ignoring -99.99 Values)
monthly_mean_temp = (
    df.loc[df["Temp_celsius"] != -99.99]  # ✅ Corrected indexing
    .groupby("Month")["Temp_celsius"]
    .mean()
)

# ✅ Replace -99.99 with the corresponding Monthly Mean
df.loc[df["Temp_celsius"] == -99.99, "Temp_celsius"] = (
    df["Month"].map(monthly_mean_temp)
)

print("✅ Replaced -99.99 Temp_celsius values with monthly means.")

✅ Replaced -99.99 Temp_celsius values with monthly means.


In [156]:
#count number of -99.99 Temp_celsius values
print(df[df['Temp_celsius'] == -99.99]['Temp_celsius'].count())

0


In [157]:
#print final data frame to a parquet file in my Google Drive
df.to_parquet("/content/drive/MyDrive/final_merged_dataset.parquet")

In [158]:
df = pd.read_parquet("/content/drive/MyDrive/final_merged_dataset.parquet")

In [160]:
df.sample(10)

,Latitude,Longitude,Observation_Date,Year,Region,Land_Cover_Description,Has_Habitat_Change,Temp_celsius,Precip_mm,Month
6520496,44.558844,-87.993751,2016-03-01,2016,US,Water,False,3.18,102.9,3
6416803,47.691618,-122.341296,2020-05-01,2020,US,Water,False,14.81,107.0,5
3703500,41.661775,-96.107022,2016-09-01,2016,US,Croplands,False,20.00,192.1,9
5977561,40.769102,-111.397934,2022-11-01,2022,US,Grasslands,False,-5.41,157.3,11
2800322,42.592648,-82.771511,2021-02-01,2021,US,Urban and Built-up,False,-3.81,11.5,2
6467348,48.089203,-119.673557,2023-03-01,2023,US,Grasslands,False,3.98,9.1,3
6132572,44.311136,-72.714858,2021-01-01,2021,US,Deciduous Broadleaf Forest,False,-7.36,49.5,1
6955023,45.335016,-86.942282,2021-03-01,2021,US,Water,False,-0.56,50.5,3
4344036,41.374286,-82.509823,2024-12-01,2024,US,Water,False,1.44,18.3,12
1792167,44.592984,-76.409884,2023-05-01,2023,US,Woody Savannas,False,12.62,43.0,5


In [161]:
df.drop(columns='Month')

,Latitude,Longitude,Observation_Date,Year,Region,Land_Cover_Description,Has_Habitat_Change,Temp_celsius,Precip_mm
0,19.343703,-96.464161,2017-10-01,2017,Mexico,Cropland/Natural Vegetation Mosaic,False,27.00,234.0
1,19.343703,-96.464161,2017-10-01,2017,Mexico,Cropland/Natural Vegetation Mosaic,False,27.00,234.0
2,19.343703,-96.464161,2017-10-01,2017,Mexico,Cropland/Natural Vegetation Mosaic,False,27.00,234.0
3,19.343703,-96.464161,2017-10-01,2017,Mexico,Cropland/Natural Vegetation Mosaic,False,27.00,234.0
4,19.343703,-96.464161,2017-10-01,2017,Mexico,Cropland/Natural Vegetation Mosaic,False,27.00,234.0
...,...,...,...,...,...,...,...,...,...
7498831,71.314808,-156.721344,2018-06-01,2018,Canada,Water,False,0.85,8.4
7498832,71.315812,-156.688557,2022-06-01,2022,Canada,Water,False,3.50,11.6
7498833,71.326271,-156.674480,2018-09-01,2018,Canada,Water,False,1.49,11.8
7498834,71.343479,-156.617538,2016-06-01,2016,Canada,Permanent Wetlands,False,1.95,18.0


In [162]:
#mean of temp_celsius by year
df.groupby('Year')['Temp_celsius'].mean()

,Temp_celsius
Year,
1974,8.021643
1975,7.541316
1976,9.286320
1977,8.794955
1978,8.034182
1979,8.122915
1980,8.881695
1981,6.363286
1982,8.922220


In [163]:
df.describe()

,Latitude,Longitude,Observation_Date,Year,Temp_celsius,Precip_mm,Month
count,7.498836e+06,7.498836e+06,7498836,7.498836e+06,7.498836e+06,7.498836e+06,7.498836e+06
mean,4.238773e+01,-9.358986e+01,2019-06-21 17:59:32.958363136,2.019055e+03,9.935046e+00,8.072112e+01,5.997464e+00
min,1.934370e+01,-1.790633e+02,1974-01-01 00:00:00,1.974000e+03,-2.975000e+01,0.000000e+00,1.000000e+00
25%,3.912949e+01,-1.140327e+02,2017-05-01 00:00:00,2.017000e+03,3.850000e+00,3.600000e+01,3.000000e+00
50%,4.240090e+01,-8.596323e+01,2020-11-01 00:00:00,2.020000e+03,9.941860e+00,6.760000e+01,5.000000e+00
75%,4.621175e+01,-7.676376e+01,2023-01-01 00:00:00,2.023000e+03,1.640000e+01,1.080000e+02,9.000000e+00
max,7.315889e+01,1.792826e+02,2024-12-01 00:00:00,2.024000e+03,3.930000e+01,1.323200e+03,1.200000e+01
std,6.207601e+00,2.055219e+01,NaN,5.087835e+00,8.348045e+00,6.630785e+01,3.617435e+00
